In [ ]:
!pip install git+https://gitlab.com/nannos/nannos.git

In [ ]:
import nannos as nn

def run(case, nh, verbose=False):
    if case == "torch cpu":
        nn.set_backend("torch")
        nn.use_gpu(False)
    elif case == "torch gpu":
        nn.set_backend("torch")
        nn.use_gpu(True)
    else:
        nn.set_backend("numpy")
        nn.use_gpu(False)
    if verbose:
        print(f">>> Exectuting with {case}...")
    bk = nn.backend
    Nx = 2**9
    lat = nn.Lattice(((1,0),(0,1)),discretization=Nx)
    epsgrid = lat.ones() * 12.0
    hole = lat.circle((0.5, 0.5), 0.2)
    epsgrid[hole] = 1.0
    sup = lat.Layer(epsilon=1, name="sup")
    sub = lat.Layer(epsilon=1, name="sub")
    struct = lat.Layer(thickness=0.5, name="struct")
    struct.epsilon=epsgrid
    layers = [sup, struct, sub]
    pw = nn.PlaneWave(wavelength=1 / 1.4, angles=(0, 0, 0))
    sim = nn.Simulation(layers, pw, nh)
    Ri, Ti = sim.diffraction_efficiencies(orders=True)
    if verbose:
        R = sum(Ri)
        T = sum(Ti)
        nmax = 5
        print("Ri = ", Ri[:nmax])
        print("Ti = ", Ti[:nmax])
        print("R = ", R)
        print("T = ", T)
        print("R + T = ", R + T)

In [ ]:
num_harmo=[50,100]#,200,300,500,800]
timings_gpu_torch=[]
timings_cpu_torch=[]
timings_numpy=[]
for nh in num_harmo:
    print(f"Number of harmonics: {nh}")
    print("------------------------")
    print("torch gpu")
    result_gpu_torch = %timeit -o run("torch gpu",nh)
    t_gpu_torch = result_gpu_torch.best
    
    print("torch cpu")
    result_cpu_torch = %timeit -o run("torch cpu",nh)
    t_cpu_torch = result_cpu_torch.best
    
    print("numpy")
    result_cpu_numpy = %timeit -o run("numpy",nh)
    t_cpu_numpy = result_cpu_numpy.best
    print()
    
    print(f"speedup torch cpu vs numpy cpu: {t_cpu_numpy/t_cpu_torch}")
    print(f"speedup torch gpu vs torch cpu: {t_cpu_torch/t_gpu_torch}")
    print(f"speedup torch gpu vs numpy cpu: {t_cpu_numpy/t_gpu_torch}")
    print()
    
    timings_gpu_torch.append(result_gpu_torch)
    timings_cpu_torch.append(result_cpu_torch)
    timings_numpy.append(result_cpu_numpy)

In [ ]:
import numpy as np
np.savez("results_kaggle.npz",num_harmo=num_harmo,timings_gpu_torch=timings_gpu_torch,timings_cpu_torch=timings_cpu_torch,timings_numpy=timings_numpy)
